In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.03_tbl_account_prism_adjustments AS
INSERT OVERWRITE TABLE teamconnect.pca_prism.03_tbl_account_prism_adjustments
WITH adj_raw AS (
  SELECT
    RIGHT(LPAD(REGEXP_REPLACE(CAST(TN AS STRING),'[^0-9]',''),10,'0'),10) AS btn,
    clean_date(TO_DATE(Post_Date)) AS post_date,
    CAST(Adj_Amt AS DECIMAL(18,2)) AS adj_amt,
    Credit_Code, Credit_Code_Description, LOBType
  FROM teamconnect.cia_interfacemaintenance.tbl_adjustments
  WHERE TN IS NOT NULL AND Post_Date IS NOT NULL
),
adj_roll AS (
  SELECT btn, COUNT(*) AS adjustments_count_all_time,
         COALESCE(SUM(adj_amt),0) AS adjustments_sum_all_time
  FROM adj_raw GROUP BY 1
),
adj_last12_src AS (
  SELECT * FROM adj_raw
  WHERE post_date >= ADD_MONTHS(current_date(), -12)
),
adj_last12 AS (
  SELECT btn,
         COLLECT_LIST(named_struct(
           'post_date',post_date,'amount',adj_amt,'credit_code',Credit_Code,'credit_desc',Credit_Code_Description
         )) AS adjustments_last_12m
  FROM adj_last12_src
  GROUP BY 1
),
adj_cur_month_src AS (
  SELECT * FROM adj_raw
  WHERE post_date >= DATE_TRUNC('month', current_date())
    AND post_date <  ADD_MONTHS(DATE_TRUNC('month', current_date()), 1)
),
adj_cur_month AS (
  SELECT btn,
         COALESCE(SUM(adj_amt),0) AS adjustments_current_month_sum,
         COLLECT_LIST(named_struct(
           'post_date',post_date,'amount',adj_amt,'credit_code',Credit_Code,'credit_desc',Credit_Code_Description
         )) AS adjustments_current_month_items
  FROM adj_cur_month_src
  GROUP BY 1
),
adj_lob_last AS (
  SELECT btn, LOBType, post_date,
         ROW_NUMBER() OVER (PARTITION BY btn ORDER BY post_date DESC) AS rn
  FROM adj_raw
  WHERE LOBType IS NOT NULL
),
adj_lob_mode_12m AS (
  SELECT btn, LOBType, cnt,
         ROW_NUMBER() OVER (PARTITION BY btn ORDER BY cnt DESC, LOBType ASC) AS rn
  FROM (
    SELECT btn, LOBType, COUNT(*) AS cnt
    FROM adj_last12_src
    WHERE LOBType IS NOT NULL
    GROUP BY btn, LOBType
  ) x
),
adj_lob AS (
  SELECT COALESCE(l.btn,m.btn) AS btn,
         MAX(CASE WHEN l.rn=1 THEN l.LOBType END) AS lobtype_last_seen,
         MAX(CASE WHEN m.rn=1 THEN m.LOBType END) AS lobtype_mode_12m
  FROM adj_lob_last l
  FULL OUTER JOIN adj_lob_mode_12m m ON m.btn=l.btn
  GROUP BY COALESCE(l.btn,m.btn)
)
SELECT
  b.btn,
  COALESCE(ar.adjustments_count_all_time, 0)        AS adjustments_count_all_time,
  COALESCE(ar.adjustments_sum_all_time,   0)        AS adjustments_sum_all_time,
  COALESCE(ar12.adjustments_last_12m, ARRAY())      AS adjustments_last_12m,
  COALESCE(arcm.adjustments_current_month_sum, 0)   AS adjustments_current_month_sum,
  COALESCE(arcm.adjustments_current_month_items, ARRAY()) AS adjustments_current_month_items,
  adjlob.lobtype_last_seen,
  adjlob.lobtype_mode_12m
FROM (
  SELECT DISTINCT btn
  FROM adj_raw
) b
LEFT JOIN adj_roll      ar     ON ar.btn   = b.btn
LEFT JOIN adj_last12    ar12   ON ar12.btn = b.btn
LEFT JOIN adj_cur_month arcm   ON arcm.btn = b.btn
LEFT JOIN adj_lob       adjlob ON adjlob.btn = b.btn;

OPTIMIZE teamconnect.pca_prism.03_tbl_account_prism_adjustments
ZORDER BY (btn);
